## <center>3D B&eacute;zier curve as spine for a ribbon surface</center>

In [1]:
using LinearAlgebra
using PlotlyJS
include("src/plotlyju.jl");

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-14759434395813284781\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-242912965100480436\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-14759434395813284781\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

De Casteljau algorithm to evaluate a point, and tangent at that point,  on a B&eacute;zier curve of control polygon, b[1], b[2],..., b[n]:

In [2]:
function deCasteljau(b::Vector{Vector{Float64}}, t::Float64)
    # b[1], ..., b[n], Bezier control points
    # t  in [0,1] the parameter to evaluate the curve parameterization 
    # returns the point on the Bezier curve corresponding to the parameter t,
    # and the tangent vector at that point
    n = size(b,1)
    n>=2 &&  0 <= t <= 1 ||  
         error("The  control polygon must have at least two points, and t∈ [0,1]")

    a = copy(b)
    for r in 1:n-2
        for i in 1:n-r
            a[i] = (1-t)*a[i] + t*a[i+1]
        end
    end       
    tangent = a[2]-a[1]   
    return (1-t)*a[1] + t*a[2],  tangent/norm(tangent) # curve point, and  unit tangent vector
end 


deCasteljau (generic function with 1 method)

In [3]:
function rotation_min_frame(cpoint::Array{Float64, 2}, T::Array{Float64, 2})
    
    #implementation of the algorithm from https://dl.acm.org/doi/10.1145/1330511.1330513
    # cpoint: array of points on a 3d curve, 
    # T: tangents at the points, cpoint
    # returns the vectors as rows in  R, and S such that
    # (R,S,T) is an  orthonormal mobile frame along the curve, of points cpoint
    
    N, D = size(cpoint) # N points in \mathbb{R}^D on a digitized  curve
    (size(T, 1) == N)  && (size(T, 2) == D) || 
         error("The arrays cpoint  and T must have the same size")
    R = zeros(N, D) #   a row in R is an r-vector in  article
    S = zeros(N, D) #  s-vectors in  article
    r, s = eachcol(nullspace(permutedims(T[1, :]))) # (r, s, T[1, :])  orthonormal basis at cpoint[1, :]
    R[1, :] = r
    S[1, :] = s
    for i = 1:N-1
        v₁ = cpoint[i+1, :] - cpoint[i, :]
        c₁ = dot(v₁, v₁)
        rL = R[i, :] - (2/c₁)*dot(v₁, R[i, :])*v₁
        tL = T[i, :] - (2/c₁)*dot(v₁, T[i, :])*v₁
        v₂ = T[i+1, :] - tL
        c₂ = dot(v₂, v₂)
        R[i+1, :] = rL - (2/c₂*dot(v₂, rL)*v₂)
        S[i+1, :] =  cross(T[i+1, :], R[i+1, :])
    end
    return R, S
end    

rotation_min_frame (generic function with 1 method)

Let us define and plot a B&eacute;zier curve with an almost linear segment with an end at b[1]:

In [4]:
b = [[1, 0, 0], [0.5, 2.4, 1], [-0.4, 6.72, 2.8], [0.75, 5, -1], [1, 4, 1]]  #Bezier  control polygon
D = length(b[1])
N = 150
t = LinRange(0,1, N)
cpoint =  zeros(N, D);
T  = zeros(N, D)
for (k, s) in enumerate(t)
    cpoint[k, :], T[k, :] = deCasteljau(b, s)
end
bx = [p[1] for p in b]
by = [p[2] for p in b]
bz = [p[3] for p in b]

tr1 = scatter3d(x=bx, y=by, z= bz, mode="markers+lines", name="control polygon", 
              line_color="RoyalBlue", marker_size=4, line_width=2.5)
tr2 = scatter3d(x=cpoint[:, 1], y=cpoint[:, 2], z=cpoint[:, 3], mode="lines", 
                name="Bezier curve", line_color="red", line_width=2)

layout = Layout(title_text="3D B\u00E9zier curve", title_x=0.5,
                             width=800, height=600, font_size=11,
                             scene_camera_eye=attr(x=1.6, y=1.6, z=1))
pl = Plot([tr1, tr2], layout, style=plotlyju)

data: [
  "scatter3d with fields colorbar, colorscale, line, marker, mode, name, type, x, y, and z",
  "scatter3d with fields colorbar, colorscale, line, marker, mode, name, type, x, y, and z"
]

layout: "layout with fields coloraxis, font, height, hovermode, margin, paper_bgcolor, plot_bgcolor, scene, ternary, title, width, xaxis, and yaxis"

Let us define a ribbon with the above B&eacute;zier curve as spine.

We implement the algorithm which  sets up  a mobile orthonormal frame along  a $C^1$ curve (not necessarily a biregular $C^2$-one, as it is required to define the Frenet frame), presented in:
W Wang, B Juttler, D Zheng and Y Liu, *Computation of Rotation Minimizing Frame*,  ACM Transactions on Graphics, Vol 27,
Issue 1, 2008, pp 1–18.

Ribbon surface definition:

In [5]:
a = 0.075  # 2a is the ribbon width
u = LinRange(-a, a, 10)
R, S = rotation_min_frame(cpoint, T);
xrib = cpoint[:, 1] * ones(size(u))'+ R[:,1]* u'
yrib = cpoint[:, 2] * ones(size(u))'+ R[:,2]* u'
zrib = cpoint[:, 3] * ones(size(u))'+ R[:,3]* u';

ribbon = surface(x=xrib, y=yrib, z=zrib, showscale=false)
fig = plot(ribbon, layout, style=plotlyju)
relayout!(fig, title_text="Ribbon with a B\u00E9zier curve as spine")

In [6]:
fig.plot

data: [
  "surface with fields colorbar, colorscale, marker, showscale, type, x, y, and z"
]

layout: "layout with fields coloraxis, font, height, hovermode, margin, paper_bgcolor, plot_bgcolor, scene, ternary, title, width, xaxis, and yaxis"